In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
from openfold.utils.loss import lddt_loss, AlphaFoldLoss
import openfold
import torch

from plaid.proteins import LatentToStructure, LatentToSequence
from plaid.esmfold.misc import batch_encode_sequences
from plaid.transforms import trim_or_pad_batch_first

print(openfold.__file__)

/homefs/home/lux70/openfold/openfold/__init__.py


# Load ground truth features and predictions
## Ground truth dictionary

In [17]:
from plaid.datasets import CATHStructureDataModule

shard_dir = "/homefs/home/lux70/storage/data/cath/shards/"
pdb_dir = "/data/bucket/lux70/data/cath/dompdb"
# shard_dir = "/homefs/home/lux70/storage/data/rocklin/shards/"
# pdb_dir = "/data/bucket/lux70/data/rocklin/structures/"

max_seq_len=256
dm = CATHStructureDataModule(
    shard_dir,
    pdb_dir,
    seq_len=max_seq_len,
    batch_size=4,
    max_num_samples=4,
    shuffle_val_dataset=False
) 
    
dm.setup()
val_dataloader = dm.val_dataloader()
next_batch = next(iter(val_dataloader))
print(len(val_dataloader.dataset))

batch = next_batch[-1]  # struct dict

4


In [18]:
batch.keys()

dict_keys(['aatype', 'between_segment_residues', 'domain_name', 'residue_index', 'seq_length', 'sequence', 'all_atom_positions', 'all_atom_mask', 'resolution', 'is_distillation', 'mask', 'rigidgroups_gt_frames', 'rigidgroups_gt_exists', 'rigidgroups_group_exists', 'rigidgroups_group_is_ambiguous', 'rigidgroups_alt_gt_frames', 'backbone_rigid_tensor', 'backbone_rigid_mask'])

In [39]:
device = torch.device("cuda")

for k, v in batch.items():
    try:
        batch[k] = v.to(device)
    except:
        print(k)
        pass

for k, v in batch.items():
    try:
        print(v.device)
    except:
        print(k); pass

domain_name
sequence
cuda:0
cuda:0
domain_name
cuda:0
cuda:0
sequence
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0


## Prediction output dictionary 

In [10]:
# get saved embedding
device = torch.device("cuda")
x = batch[0].to(device)
sequences = batch[1]

# get corresponding mask, to be used later
_, mask, _, _, _ = batch_encode_sequences(sequences)
mask = trim_or_pad_batch_first(mask, pad_to=max_seq_len, pad_idx=0).to(device)

# implicitly calls Openfold structure module
latent_to_structure = LatentToStructure()
latent_to_structure.to(device)
orig_struct = latent_to_structure.to_structure(x, sequences, return_raw_features=True, batch_size=4, num_recycles=1)

(Generating structure): 100%|███████████████████████████████████████████████| 1/1 [00:09<00:00,  9.34s/it]


In [15]:
out = orig_struct[-1][0]
out.keys()

dict_keys(['frames', 'sidechain_frames', 'unnormalized_angles', 'angles', 'positions', 'states', 's_initial', 's_after_layernorm', 'z_after_layernorm', 'single', 'sm_s', 'sm_z', 's_s', 's_z', 'distogram_logits', 'lm_logits', 'aatype', 'atom14_atom_exists', 'residx_atom14_to_atom37', 'residx_atom37_to_atom14', 'atom37_atom_exists', 'residue_index', 'lddt_head', 'plddt', 'ptm_logits', 'ptm', 'aligned_confidence_probs', 'predicted_aligned_error', 'max_predicted_aligned_error'])

In [27]:
from openfold.np import residue_constants
residue_constants.atom_order['CA']

1

## Examine key shapes 

In [24]:
print(out['positions'].shape)

torch.Size([8, 4, 256, 14, 3])


In [26]:
print(batch['all_atom_positions'].shape)
print(batch['all_atom_mask'].shape)

torch.Size([4, 256, 37, 3])
torch.Size([4, 256, 37])


## Get loss config

In [11]:
from openfold.config import model_config
config = model_config(name="initial_training", train=True)

In [12]:
config.loss

distogram:
  eps: &id001 !!python/object:ml_collections.config_dict.config_dict.FieldReference
    _field_type: !!python/name:builtins.float ''
    _ops: []
    _required: false
    _value: 1.0e-08
  max_bin: 21.6875
  min_bin: 2.3125
  no_bins: 64
  weight: 0.3
eps: *id001
experimentally_resolved:
  eps: *id001
  max_resolution: 3.0
  min_resolution: 0.1
  weight: 0.0
fape:
  backbone:
    clamp_distance: 10.0
    loss_unit_distance: 10.0
    weight: 0.5
  eps: 0.0001
  sidechain:
    clamp_distance: 10.0
    length_scale: 10.0
    weight: 0.5
  weight: 1.0
masked_msa:
  eps: *id001
  weight: 2.0
plddt_loss:
  cutoff: 15.0
  eps: *id001
  max_resolution: 3.0
  min_resolution: 0.1
  no_bins: 50
  weight: 0.01
supervised_chi:
  angle_norm_weight: 0.01
  chi_weight: 0.5
  eps: *id001
  weight: 1.0
tm:
  enabled: !!python/object:ml_collections.config_dict.config_dict.FieldReference
    _field_type: !!python/name:builtins.bool ''
    _ops: []
    _required: false
    _value: false
  eps: *

# Try Losses

In [19]:
from openfold.utils.loss import distogram_loss
distogram_loss(out['distogram_logits'], **{**batch, **config.loss.distogram})

TypeError: distogram_loss() missing 2 required positional arguments: 'pseudo_beta' and 'pseudo_beta_mask'

In [20]:
from openfold.utils.loss import experimentally_resolved_loss
experimentally_resolved_loss(
    logits=out['experimentally_resolved_logits'],
    **{**batch, **config.loss.experimentally_resolved}
)

KeyError: 'experimentally_resolved_logits'

In [40]:
from openfold.utils.loss import fape_loss
wrapper_out = {"sm": out}
fape_loss(wrapper_out, batch, config.loss.fape)

TypeError: sidechain_loss() missing 3 required positional arguments: 'renamed_atom14_gt_positions', 'renamed_atom14_gt_exists', and 'alt_naming_is_better'

In [41]:
from openfold.utils.loss import lddt_loss
lddt_loss(
    logits=out['plddt'].to(device), #TODO: these are not logits
    all_atom_pred_pos=out['positions'].to(device),  #TODO: confim if these are the same
    **{**batch, **config.loss.plddt_loss}
)
out.keys()

RuntimeError: The size of tensor a (50) must match the size of tensor b (37) at non-singleton dimension 3

In [ ]:
%debug

> /homefs/home/lux70/openfold/openfold/utils/loss.py(39)softmax_cross_entropy()
     37 def softmax_cross_entropy(logits, labels):
     38     loss = -1 * torch.sum(
---> 39         labels * torch.nn.functional.log_softmax(logits, dim=-1),
     40         dim=-1,
     41     )



ipdb>  up


> /homefs/home/lux70/openfold/openfold/utils/loss.py(503)lddt_loss()
    501     )
    502 
--> 503     errors = softmax_cross_entropy(logits, lddt_ca_one_hot)
    504     all_atom_mask = all_atom_mask.squeeze(-1)
    505     loss = torch.sum(errors * all_atom_mask, dim=-1) / (



ipdb>  l


    498     bin_index = torch.clamp(bin_index, max=(no_bins - 1))
    499     lddt_ca_one_hot = torch.nn.functional.one_hot(
    500         bin_index, num_classes=no_bins
    501     )
    502 
--> 503     errors = softmax_cross_entropy(logits, lddt_ca_one_hot)
    504     all_atom_mask = all_atom_mask.squeeze(-1)
    505     loss = torch.sum(errors * all_atom_mask, dim=-1) / (
    506         eps + torch.sum(all_atom_mask, dim=-1)
    507     )
    508 



In [21]:
from openfold.utils.loss import supervised_chi_loss
supervised_chi_loss(
    out["angles"], out["unnormalized_angles"], **{**batch, **config.loss.supervised_chi}
)

TypeError: supervised_chi_loss() missing 3 required positional arguments: 'seq_mask', 'chi_mask', and 'chi_angles_sin_cos'

In [22]:
from openfold.utils.loss import violation_loss
violation_loss(
    out['violation'], **{**batch, **config.loss.violation}
)

KeyError: 'violation'

In [23]:
from openfold.utils.loss import tm_loss
tm_loss(
    logits=out['ptm_logits'], 
    **{**batch, **out, **config.loss.tm}
)

TypeError: tm_loss() missing 1 required positional argument: 'final_affine_tensor'

In [99]:
from openfold.utils.loss import chain_center_of_mass_loss
chain_center_of_mass_loss(
    all_atom_pred_pos=out['positions'],
    **{**batch, **config.loss.chain_center_of_mass}
)

ImportError: cannot import name 'chain_center_of_mass_loss' from 'openfold.utils.loss' (/homefs/home/lux70/openfold/openfold/utils/loss.py)